# Antenna Buffer commands, guide and examples
The antenna buffer serves the role of stopping information captured from the antennas for a short period of time. The buffer control methods are exposed through the tile device.

To use the commands, first all the devices should be on:

In [6]:
import tango
import time
import json

db = tango.Database()
all_devices_strings = db.get_device_exported("*")
recheck = False
for device_str in all_devices_strings:
    device = tango.DeviceProxy(device_str)
    if device.state() == tango.DevState.DISABLE:
        print(f"{device_str} is in {device.state()} state, turning it ON")
        device.adminMode = 0
        recheck = True
    else:
        print(f"{device_str} is in {device.state()} state")

if recheck:
    for i in range(30):
        print(f"Starting devices in progress {i/0.3:.2f}%", end="")
        time.sleep(1)
        print("\r", end="")  # put the cursor back on the same line
    
    for device_str in all_devices_strings:
        device = tango.DeviceProxy(device_str)
        print(f"{device_str} is in {device.state()} state")


dserver/DataBaseds/2 is in ON state
dserver/TangoTest/test is in ON state
dserver/TestDevice/test is in ON state
sys/database/2 is in ON state
sys/tg_test/1 is in RUNNING state
test/device/1 is in UNKNOWN state


## Setting up the antenna buffer

Before recording the antenna buffer, it needs to be initiated.

The set up values used are:

- mode: the location where the data will be read to. Either SDN for the science data netwrok, or NSDN (non science data network). Defaults to SDN
- ddr_start_byte_address: first address (in bytes) in the DDR to write to. Defaults to 512 MiB
- max_ddr_byte_size: last address. If None is chosen, then it uses all the remaining space.



In [4]:
tile = tango.DeviceProxy("low-mccs/tile/ci-1-tpm10")

setup_values = {
    "mode": "SDN",
    "ddr_start_byte_address": 0,
    "max_ddr_byte_size": None
}
code, message = tile.SetUpAntennaBuffer(json.dumps(setup_values))
print(message)

DevFailed: DevFailed[
DevError[
    desc = device low-mccs/tile/ci-1-tpm10 not defined in the database !
  origin = DataBase::ImportDevice()
  reason = DB_DeviceNotDefined
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device sys/database/2, command DbImportDevice
  origin = virtual Tango::DeviceData Tango::Connection::command_inout(const string&, const Tango::DeviceData&) at (/home/conda/feedstock_root/build_artifacts/cpptango_1675413853047/work/cppapi/client/devapi_base.cpp:1338)
  reason = API_CommandFailed
severity = ERR]

DevError[
    desc = Can't connect to device low-mccs/tile/ci-1-tpm10
  origin = void Tango::DeviceProxy::real_constructor(const string&, bool) at (/home/conda/feedstock_root/build_artifacts/cpptango_1675413853047/work/cppapi/client/devapi_base.cpp:1705)
  reason = API_DeviceNotDefined
severity = ERR]
]

## Starting the antenna buffer

Once the buffer is set up, the user can start writing.

The write values used are:

- Antennas: a list of antennas IDs to be used. There are 16 antennas and the IDs range from 0 to 15. The antennas are separated in FPGAs, of which there are 2. The first FPGA has antennas 0-7 and the second one has 8-15. Only a maximum of two antennas can be specified for each FPGA. There is no default value here, the software will return an error if the input is missing or incorrect. 
- start_time: The first timestamp to be written. If the value is -1, then it will start writing as soon as possible. Defaults to -1.
- timestamp_capture_duration: capture duration in timestamps. Defaults to 75
- continuous_mode: if enabled, timestamp duration is ignored. Defaults to False

In [ ]:
start_values = {
    "antennas": [1,2],
    "start_time": 0,
    "timestamp_capture_duration": None,
    "continuous_mode": True,
}
code, message = tile.StartAntennaBuffer(json.dumps(start_values))
print(message)

DevFailed: DevFailed[
DevError[
    desc = ValueError: Antennas list is empty, please provide at least one antenna id: {}
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/server.py", line 1771, in wrapped_command_method
    return get_worker().execute(cmd_method, self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 113, in execute
    return fn(*args, **kwargs)
  File "/app/src/ska_low_mccs_spshw/tile/tile_device.py", line 5811, in StartAntennaBuffer
    (return_code, message) = handler(argin)
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/commands.py", line 250, in __call__
    return self.invoke(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/commands.py", line 317, in invoke
    return self.do(*args, **kwargs)
  File "/app/src/ska_low_mccs_spshw/tile/tile_device.py", line 5771, in do
    raise ValueError(f"{err}: {kwargs}")
ValueError

## Read the antenna buffer

Reads from the antenna buffer to the DAQ reciever. Stops continous mode.

In [ ]:
code, message = tile.ReadAntennaBuffer()
print(message)

DevFailed: DevFailed[
DevError[
    desc = KeyError: 'data_capture_initiated'
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/server.py", line 1771, in wrapped_command_method
    return get_worker().execute(cmd_method, self, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 113, in execute
    return fn(*args, **kwargs)
  File "/app/src/ska_low_mccs_spshw/tile/tile_device.py", line 5862, in ReadAntennaBuffer
    (return_code, message) = handler()
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/commands.py", line 250, in __call__
    return self.invoke(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/commands.py", line 317, in invoke
    return self.do(*args, **kwargs)
  File "/app/src/ska_low_mccs_spshw/tile/tile_device.py", line 5844, in do
    self._component_manager.read_antenna_buffer()
  File "/usr/local/lib/python3.10/dist-packages/ska

## Stop the antenna buffer

Stops the antenna buffer

In [ ]:
start_values = {
    "antennas": [1,2],
    "start_time": 0,
    "timestamp_capture_duration": None,
    "continuous_mode": True,
}
code, message = tile.StartAntennaBuffer(json.dumps(start_values))

code, message = tile.StopAntennaBuffer()
print(message)

[array([0], dtype=int32), ['StopAntennaBuffer command completed OK']]
